<a href="https://colab.research.google.com/github/Kridr/HW_PB_HSE/blob/main/HW5_PB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.4 Установка hmmer

Скачивание



In [ ]:
!wget eddylab.org/software/hmmer/hmmer.tar.gz  

--2022-10-28 21:45:35--  http://eddylab.org/software/hmmer/hmmer.tar.gz
Resolving eddylab.org (eddylab.org)... 96.126.110.11, 2600:3c03::f03c:91ff:fec8:383c
Connecting to eddylab.org (eddylab.org)|96.126.110.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18213049 (17M) [application/x-gzip]
Saving to: ‘hmmer.tar.gz.4’

hmmer.tar.gz.4      100%[===================>]  17.37M  73.1MB/s    in 0.2s    

2022-10-28 21:45:35 (73.1 MB/s) - ‘hmmer.tar.gz.4’ saved [18213049/18213049]



Распаковка

In [ ]:
!tar xf hmmer.tar.gz

Установка 

In [ ]:
!hmmer-3.3.2/configure
!make
!make check

configure: Configuring HMMER3 for your system.
checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking whether to compile using MPI... no
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
checking for gcc option to accept ISO C89... none needed
checking for gcc option to accept ISO C99... none needed
checking for gcc option to accept ISO Standard C... (cached) none needed
checking how to run the C preprocessor... gcc -E
checking for a BSD-compatible install... /usr/bin/install -c
checking for strip... strip
checking for ranlib... ranlib
checking for ar... /usr/bin/ar
checking whether ln -s works... yes
checking for a sed that does not truncat

# 1.5 Запуск hmmer на всех выравниваниях

Для каждого файла выравнивания делаем следующее:
1. Все выравнивания изначально запакованы в .gz и их нужно распаковать (мне захотелось сохранить в отдельную папку).
2. Используем `hmmbuild`, после установки он лежит в отдельной папке src
3. Распаковываем белок из .gz формата
4. Используем `hmmsearch` и записываем соответсвующий выравниванию результат в файл в папке

In [ ]:
import os

for fn in os.listdir('alignments'):
    print(fn, '...')
    fn_hmm = fn.split('.')[0]
    r1 = os.system(f'gunzip -c alignments/{fn} > alignments_dc/{fn[:-3]}')
    r2 = os.system(f'src/hmmbuild profiles/{fn_hmm}.hmm alignments_dc/{fn[:-3]}')
    r3 = os.system(f'gunzip -c sd_protein.faa.gz > sd_protein.faa')
    r4 = os.system(f'src/hmmsearch profiles/{fn_hmm}.hmm sd_protein.faa > results/{fn_hmm}.txt')

PF07730.alignment.seed.gz ...
PF02518.alignment.seed.gz ...
PF01295.alignment.seed.gz ...
PF00211.alignment.seed.gz ...
PF01966.alignment.seed.gz ...
PF00015.alignment.seed.gz ...
PF00563.alignment.seed.gz ...
PF01928.alignment.seed.gz ...
PF00512.alignment.seed.gz ...
PF07536.alignment.seed.gz ...
PF00069.alignment.seed.gz ...
PF00990.alignment.seed.gz ...
PF07568.alignment.seed.gz ...


Ниже код для запаковки папки с результатами. Colab не дает скачать просто папку

In [ ]:
!zip -r results.zip results

  adding: results/ (stored 0%)
  adding: results/PF01966.txt (deflated 70%)
  adding: results/PF01295.txt (deflated 56%)
  adding: results/PF00990.txt (deflated 71%)
  adding: results/PF07536.txt (deflated 58%)
  adding: results/PF00015.txt (deflated 60%)
  adding: results/PF00069.txt (deflated 56%)
  adding: results/PF02518.txt (deflated 77%)
  adding: results/PF01928.txt (deflated 59%)
  adding: results/PF07730.txt (deflated 73%)
  adding: results/.ipynb_checkpoints/ (stored 0%)
  adding: results/PF07568.txt (deflated 58%)
  adding: results/PF00211.txt (deflated 58%)
  adding: results/PF00563.txt (deflated 70%)
  adding: results/PF00512.txt (deflated 77%)


# 1.6 Анализ результатов и подсчет сигнальных блоков

In [6]:
import pandas as pd
import numpy as np
import os.path

Распишем условия.

Условие представляет из себя список списков.

Внешний список содержит те условия, которые обязательно необходимо соблюсти. Например, для `targets_hk` нужно соблюсти два условия.

Внутри каждого условия содержатся строки. Для соблюдения условия необходимо, чтобы хотя бы одна строка содержалась в Accession (см. далее)

In [21]:
targets_hk = [['pfam00512', 'pfam07730'], ['pfam02518']]
targets_mac = [['pfam00015']]
targets_k = [['pfam00069']]
targets_dc = [['pfam00990']]
targets_ac = [['pfam01295', 'pfam01928']]
targets_gmp = [['pfam00563', 'pfam01966']]


correspond_dict = {
    'PF00512': targets_hk,
    'PF07568': targets_hk,
    'PF07730': targets_hk,
    'PF07536': targets_hk,
    'PF02518': targets_hk,
    'PF00015': targets_mac,
    'PF00069': targets_k,
    'PF00990': targets_dc,
    'PF01295': targets_ac,
    'PF01928': targets_ac,
    'PF00211': targets_ac,
    'PF00563': targets_gmp,
    'PF01966': targets_gmp
}

Здесь я ищу все уникальные находки и для каждой находки с помощью поля Accession проверяю все уникальные находки на соблюдение условия

In [24]:
records = []
for domain, target in correspond_dict.items():
    if os.path.isfile(f'hitdata/hitdata_{domain}.txt'):
        df_temp = pd.read_csv(f'hitdata/hitdata_{domain}.txt', sep='\t', skiprows=7)
        queries = df_temp['Query'].unique()

        succ_num = 0
        for query in queries:
            s = set(df_temp[df_temp['Query'] == query]['Accession'])
            succ = True
            for subtargets in target:
                sub_succ = False
                for subtarget in subtargets:
                    sub_succ = sub_succ or (subtarget in s)
                succ = succ and sub_succ
            if succ:
                succ_num += 1
        
        records.append((domain, succ_num, len(queries)))              
    else:
        records.append((domain, 0, 0))

df_res = pd.DataFrame.from_records(records, columns=['Domain', 'Good', 'All'])
df_res

,Domain,Good,All
0,PF00512,17,17
1,PF07568,0,0
2,PF07730,3,4
3,PF07536,0,0
4,PF02518,20,29
5,PF00015,1,1
6,PF00069,1,1
7,PF00990,6,6
8,PF01295,1,1
9,PF01928,1,1


# 1.7 Подсчет IQ

Подсчитываем прямо по определению: делим количество сигнальных белков на общее количество белков.

In [25]:
round(df_res['Good'].sum() / df_res['All'].sum(), 4)

0.8333

In [26]:
print(df_res.to_latex(index=False))

\begin{tabular}{lrr}
\toprule
 Domain &  Good &  All \\
\midrule
PF00512 &    17 &   17 \\
PF07568 &     0 &    0 \\
PF07730 &     3 &    4 \\
PF07536 &     0 &    0 \\
PF02518 &    20 &   29 \\
PF00015 &     1 &    1 \\
PF00069 &     1 &    1 \\
PF00990 &     6 &    6 \\
PF01295 &     1 &    1 \\
PF01928 &     1 &    1 \\
PF00211 &     0 &    0 \\
PF00563 &     6 &    6 \\
PF01966 &     4 &    6 \\
\bottomrule
\end{tabular}

